<a href="https://colab.research.google.com/github/Edwin372/Thesis/blob/main/Vudenc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/LauraWartschinski/VulnerabilityDetection.git


Cloning into 'VulnerabilityDetection'...
remote: Enumerating objects: 1250, done.
remote: Counting objects: 100% (158/158), done.
remote: Compressing objects: 100% (133/133), done.
remote: Total 1250 (delta 44), reused 133 (delta 25), pack-reused 1092
Receiving objects: 100% (1250/1250), 294.55 MiB | 13.05 MiB/s, done.
Resolving deltas: 100% (473/473), done.
Checking out files: 100% (206/206), done.


In [2]:
!pip install torchmetrics

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 419 kB 26.6 MB/s 


In [3]:
!mv /content/VulnerabilityDetection/Code/w2v /content
!mv /content/VulnerabilityDetection/Code/myutils.py /content
!mv /content/VulnerabilityDetection/Code/data /content

In [4]:
!unrar e /content/w2v/othermodels.rar /content/w2v


UNRAR 5.50 freeware      Copyright (c) 1993-2017 Alexander Roshal


Extracting from /content/w2v/othermodels.rar

Extracting  /content/w2v/word2vec_withoutString10-1-200.model              0%  1%  2%  3%  OK 
Extracting  /content/w2v/word2vec_withoutString10-5-200.model              3%  4%  5%  6%  OK 
Extracting  /content/w2v/word2vec_withoutString10-10-200.model             6%  7%  8%  9% 10%  OK 
Extracting  /content/w2v/word2vec_withoutString10-30-200.model            10% 11% 12% 13%  OK 
Extracting  /content/w2v/word2vec_withoutString10-50-200.model            13% 14% 15% 16%  OK 
Extracting  /content/w2v/word2vec_withoutString10-100-200.model           16% 17% 18% 19% 20%  OK 
Extracting  /content/w2v/word2vec_withoutString10-300-200.model           20% 21% 22% 23%  OK 
Extracting  /content/w2v/word2vec_withoutString5000-1-200

In [5]:
!unrar e /content/w2v/pythontraining_withString.rar /content/w2v


UNRAR 5.50 freeware      Copyright (c) 1993-2017 Alexander Roshal


Extracting from /content/w2v/pythontraining_withString.rar

Extracting  /content/w2v/pythontraining_withString_X                       0%  1%  2%  3%  4%  5%  6%  7%  8%  9% 10% 11% 12% 13% 14% 15% 16% 17% 18% 19% 20% 21% 22% 23% 24% 25% 26% 27% 28% 29% 30% 31% 32% 33% 34% 35% 36% 37% 38% 39% 40% 41% 42% 43% 44% 45% 46% 47% 48% 49% 50% 51% 52% 53% 54% 55% 56% 57% 58% 59% 60% 61% 62% 63% 64% 65% 66% 67% 68% 69% 70% 71% 72% 73% 74% 75% 76% 77% 78% 79% 80% 81% 82% 83% 84% 85% 86% 87% 88% 89% 90% 91% 92% 93% 94% 95% 96% 97% 98% 9

In [6]:
import torch
import torch.nn as nn
import myutils
import sys
import os
from sklearn.metrics import accuracy_score 
from sklearn.metrics import precision_score 
from sklearn.metrics import recall_score 
from sklearn.metrics import f1_score 
from sklearn.utils import class_weight
from gensim.models import Word2Vec 

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [79]:
#paramerters for filtering and creation of samples
mode='sql'
model_name = 'LSTM'
progress = 0
count = 0
restriction = [20000, 5, 6, 10]
step = 5
full_length = 200
mode_param = str(step)+"_"+str(full_length)

In [80]:
#parameters for w2v model
mincount = 5000 #minimum time a word has to appear in the corpus to be in the w2vec model
iterationen = 300  #training iterations for the w2v model
s = 200 #d
w = "withString"
w2v_name = "word2vec_" + w + str(mincount) + "-" + str(iterationen) + "-" + str(s)
w2v_model = 'w2v/'+ w2v_name + ".model"

w2v_model = Word2Vec.load(w2v_model)


In [81]:
import json
all_blocks = []
with open('data/plain_' + mode, 'r') as infile:
  data = json.load(infile)


# print(data)

In [82]:

# allblocks = []
# for r in data:
#   progress = progress + 1
#   for c in data[r]:
    
#     if "files" in data[r][c]:                      
#       # if len(data[r][c]["files"]) > restriction[3]:
#       #   # too many files
#       #  continue
      
#       for f in data[r][c]["files"]:
        
#         # if len(data[r][c]["files"][f]["changes"]) >= restriction[2]:
#         #   #too many changes in a single file
#         #   continue
        
#         if not "source" in data[r][c]["files"][f]:
#           #no sourcecode
#           continue
        
#         if "source" in data[r][c]["files"][f]:
#           sourcecode = data[r][c]["files"][f]["source"]                          
#           # if len(sourcecode) > restriction[0]:
#           #   #sourcecode is too long
#           #   continue
          
#           allbadparts = []
          
#           for change in data[r][c]["files"][f]["changes"]:
#                 #get the modified or removed parts from each change that happened in the commit                  
#                 badparts = change["badparts"]
#                 count = count + len(badparts)
#                 # if len(badparts) > restriction[1]:
#                 #   #too many modifications in one change
#                 #   break
                 
                
#                 for bad in badparts:
#                   #check if they can be found within the file
#                   pos = myutils.findposition(bad,sourcecode)
#                   if not -1 in pos:
#                       allbadparts.append(bad)
#                 # if (len(allbadparts) > restriction[2]):
#                 #   # too many bad positions in the file
#                 #   break
#           if(len(allbadparts) > 0):
#             # if len(allbadparts) < restriction[2]:
#             #   # find the positions of all modified parts
#             #   positions = myutils.findpositions(allbadparts,sourcecode)

#               #get the file split up in samples
#               blocks = myutils.getblocks(sourcecode, positions, step, full_length)
              
#               for b in blocks:
#                   #each is a tuple of code and label
#                   allblocks.append(b)


In [83]:
import torch
from torch.utils.data import Dataset
from torchtext.transforms import ToTensor, PadTransform
import random
class MyDataset(Dataset):
  def __init__(self, allblocks ,data_split='train', transform=None, target_transform=None):
    self.allblocks = allblocks 
    self.transform = transform
    self.target_transform = target_transform
    self.data_split=data_split
  
    keys = []
    for i in range(len(self.allblocks)):
      keys.append(i)
    random.shuffle(keys)
    cutoff = round(0.7 * len(keys)) #     70% for the training set
    cutoff2 = round(0.85 * len(keys)) #   15% for the validation set and 15% for the final test set
    self.keystrain = keys[:cutoff]
    self.keysvalid = keys[cutoff:cutoff2]
    self.keystest = keys[cutoff2:]

    default_idx = self.keystrain[0]
    block = self.allblocks[default_idx]
    code = block[0]
    self.default_token = myutils.getTokens(code)

  def __len__(self):
    if self.data_split == 'train': 
      return len(self.keystrain)
    elif self.data_split == 'valid':
      return len(self.keysvalid)
    elif self.data_split == 'test': 
      return len(self.keystest)
    else:
      print('Invalid split')
      


  def __getitem__(self, idx):
    word_vectors = w2v_model.wv
    if self.data_split == 'train': 
      key_idx = self.keystrain[idx]
    elif self.data_split == 'valid':
      key_idx = self.keysvalid[idx]
    elif self.data_split == 'test':
      key_idx = self.keystest[idx]
    block = self.allblocks[key_idx]
    code = block[0]
    token = myutils.getTokens(code) #get all single tokens from the snippet of code
    if(len(token) == 0): 
      print(len(token))
      token = self.default_token

    vectorlist = []
    if block[1] == 1: 
      label = 0 
    else: 
      label = 1
    for t in token: #convert all tokens into their word2vec vector representation
      if t in word_vectors.vocab and t != " ":
        word_vector = w2v_model[t]
        vectorlist.append(word_vector) 
    vectorlist = torch.tensor(vectorlist)
    if self.transform:
      vectorlist = self.transform(vectorlist)
    if self.target_transform:
      label = self.target_transform(label)
    return vectorlist, label 
    

In [84]:
# from torchtext.transforms import ToTensor, PadTransform

# train_dataset = MyDataset(
#     allblocks=allblocks,
#     data_split='train',
# )

# valid_dataset = MyDataset(
#     allblocks=allblocks,
#     data_split='valid'
# )

# test_dataset = MyDataset(
#     allblocks=allblocks,
#     data_split='test'
# )



In [85]:
data_path = '/content/drive/MyDrive/data'
isFiltered = False 
name = '_filtered' if isFiltered else '' 
# torch.save(train_dataset, f'{data_path}/train_data_{mode}{name}.pth')
# torch.save(valid_dataset, f'{data_path}/valid_data_{mode}{name}.pth')
# torch.save(test_dataset, f'{data_path}/test_data_{mode}{name}.pth')

In [86]:
train_dataset = torch.load(f'{data_path}/train_data_{mode}{name}.pth')
valid_dataset = torch.load(f'{data_path}/valid_data_{mode}{name}.pth')
test_dataset = torch.load(f'{data_path}/test_data_{mode}{name}.pth')

In [87]:
print(len(train_dataset))
print(len(valid_dataset))
print(len(test_dataset))

199219
42690
42690


In [88]:
print(train_dataset[10][0].shape)
print(valid_dataset[10][0].shape)
print(test_dataset[10][0].shape)

torch.Size([59, 200])
torch.Size([37, 200])
torch.Size([40, 200])


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:61: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


In [89]:
from torch.nn.utils.rnn import pad_sequence
batch_size = 128 
class MyCollate:
    def __init__(self, pad_idx):
        self.pad_idx = pad_idx
        
    
    def __call__(self, batch):
      source = []
      target = []
      for i in range(len(batch)):
        if batch[i][0].shape[0] != 0:
          source.append(batch[i][0]) 
          target.append(batch[i][1])

      for i in range(batch_size - len(source)):
        source.append(source[-1])
        target.append(target[-1])
      
      source = pad_sequence(source, batch_first=True, padding_value = self.pad_idx) 
      return source, target 

In [90]:
from torch.utils.data import DataLoader 

def get_data_loader(dataset, batch_size, num_workers=0, shuffle=True, pin_memory=True): #increase num_workers according to CPU
    pad_idx = 0 
    loader = DataLoader(
        dataset,
        batch_size = batch_size, 
        num_workers = num_workers,
        shuffle=shuffle,
        pin_memory=pin_memory, 
        collate_fn = MyCollate(pad_idx=pad_idx)
    ) 
    return loader

In [91]:
 
train_loader = get_data_loader(train_dataset, batch_size=batch_size )
valid_loader = get_data_loader(valid_dataset, batch_size=batch_size)
test_loader = get_data_loader(test_dataset, batch_size=batch_size)

In [92]:
for step, data in enumerate(train_loader):
  print(data[0].shape)
  break

torch.Size([128, 72, 200])


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:61: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


In [70]:
cudaEnabled = True 

In [151]:
from sklearn.utils.class_weight import compute_class_weight
import numpy as np
y = np.array([])
y = np.concatenate([label for item, label in train_loader])
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y), y=y)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:61: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


In [152]:
class_weights

array([ 0.51543491, 16.69705094])

In [153]:
class MyGRUModel(nn.Module):
  def __init__(self, hidden_size=200, num_gru_layers=1, gru_bidirectional=False):
    super(MyGRUModel, self).__init__()
    self.hidden_size = hidden_size
    self.embedding = w2v_model
    self.num_gru_layers = num_gru_layers
    self.gru_bidirectional = gru_bidirectional
    self.D = 2 if gru_bidirectional else 1
    self.model= nn.GRU(
        hidden_size,
        hidden_size,
        dropout=0.2,
        batch_first=True, 
        num_layers=self.num_gru_layers,
        bidirectional=self.gru_bidirectional
    )
    self.out = nn.Linear(hidden_size, 1)
    self.sigmoid = nn.Sigmoid()
  
  def forward(self, input, hidden):
    hidden = hidden.cuda()
    output, hidden = self.model(input, hidden )
    output = self.out(output)
    output = self.sigmoid(output)
    return output, hidden
  
  def initHidden(self, batch_size=1):
    hidden = torch.zeros(self.D * self.num_gru_layers, batch_size ,self.hidden_size)
    if cudaEnabled: hidden = hidden.cuda()
    return hidden 
    


In [154]:
class MyLSTMModel(nn.Module):
  def __init__(self, hidden_size=200, num_lstm_layers=1, lstm_bidirectional=False):
    super(MyLSTMModel, self).__init__()
    self.hidden_size = hidden_size
    self.embedding = w2v_model
    self.num_lstm_layers = num_lstm_layers
    self.lstm_bidirectional = lstm_bidirectional
    self.D = 2 if lstm_bidirectional else 1
    self.model= nn.LSTM(
        hidden_size,
        hidden_size,
        dropout=0.2,
        batch_first=True, 
        num_layers=self.num_lstm_layers,
        bidirectional=self.lstm_bidirectional
    )
    self.out = nn.Linear(hidden_size, 1)
    self.sigmoid = nn.Sigmoid()
  
  def forward(self, input, hidden):
    (h0, c0) = hidden
    h0 = h0.cuda()
    c0 = c0.cuda()
    output, (hn, cn) = self.model(input, (h0, c0))
    output = self.out(output)
    output = self.sigmoid(output)
    return output, (hn, cn) 
  
  def initHidden(self, batch_size=1):
    h0 = torch.zeros(self.D * self.num_lstm_layers, batch_size ,self.hidden_size)
    c0 = torch.zeros(self.D * self.num_lstm_layers, batch_size ,self.hidden_size)
    if cudaEnabled: 
      h0 = h0.cuda()
      c0 = h0.cuda()
    return h0, c0 
   

In [155]:
model = MyGRUModel() if model_name == 'GRU' else MyLSTMModel()
if cudaEnabled: model = model.cuda()

# model.load_state_dict(torch.load(f'/content/drive/MyDrive/models/model_{model_name}_{mode}.pth'))

In [156]:
import math
import time
print_every = 1000
plot_every = 50000

def timeSince(since):
  now = time.time()
  s = now - since
  m = math.floor(s/60)
  s -= m * 60
  return f'{m}m {math.floor(s)}s' 


In [157]:
def weighted_binary_cross_entropy(output, target, weights=None):
    # print(output, target)
    if weights is not None:
        assert len(weights) == 2
        loss = weights[1] * (target * torch.log(output)) + \
               weights[0] * ((1 - target) * torch.log(1 - output))
    else:
        loss = target * torch.log(output) + (1 - target) * torch.log(1 - output)

    return torch.neg(torch.mean(loss))

In [158]:

from torchmetrics import F1Score, Precision, Recall
def evaluate(model, is_test=False):
  print(50*'-' + 'evaluating' + 50*'-')
  targets = torch.tensor([]).cuda() 
  preds = torch.tensor([]).cuda() 
  start = time.time()
  all_losses = []

  loader = test_loader if is_test else valid_loader
  
  with torch.no_grad():
    print_count = 0 
    for iter, (input , labels) in enumerate(test_loader):

      hidden = model.initHidden(batch_size=batch_size)
      label_tensor = torch.tensor(labels)

      if cudaEnabled: 
        label_tensor = label_tensor.cuda()
        input = input.cuda()

      targets = torch.cat((targets, torch.tensor(label_tensor)))
      output, hidden = model(input, hidden)
      final_output = output[:, -1, :].T
      loss = weighted_binary_cross_entropy(final_output , label_tensor, weights=class_weights)
      all_losses.append(loss)
      preds = torch.cat((preds, final_output), 1)

      if iter * batch_size > print_every * print_count:
        print_count += 1
        print(f'iter: {iter * batch_size} {math.floor(iter * batch_size/len(valid_dataset) * 100)}% ({timeSince(start)})')

    eval_loss = torch.mean(torch.tensor(all_losses))
    preds = torch.squeeze(preds)
    targets = targets.int()
  if cudaEnabled:
    preds = torch.tensor(preds).cuda()
    targets = torch.tensor(targets).cuda()
    f1 = F1Score(num_classes=1).cuda()
    precision = Precision(num_classes=1).cuda()
    recall = Recall(num_classes=1).cuda()
  else:
    preds = torch.tensor(preds)
    targets = torch.tensor(targets)
    f1 = F1Score(num_classes=1)
    precision = Precision(num_classes=1)
    recall = Recall(num_classes=1)

  f1_score = f1(preds, targets)
  precision_score = precision(preds, targets)
  recall_score = recall(preds, targets)
  print(f'f1 score: {f1_score}')
  print(f'precison score: {precision_score}')
  print(f'recall score: {recall_score}')

  return eval_loss, f1_score, precision_score, recall_score

  

In [150]:
evaluate(model)

--------------------------------------------------evaluating--------------------------------------------------


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:61: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


iter: 128 0% (0m 0s)
iter: 1024 2% (0m 1s)
iter: 2048 4% (0m 2s)
iter: 3072 7% (0m 2s)
iter: 4096 9% (0m 3s)
iter: 5120 11% (0m 4s)
iter: 6016 14% (0m 5s)
iter: 7040 16% (0m 6s)
iter: 8064 18% (0m 7s)
iter: 9088 21% (0m 8s)
iter: 10112 23% (0m 9s)
iter: 11008 25% (0m 10s)
iter: 12032 28% (0m 11s)
iter: 13056 30% (0m 12s)
iter: 14080 32% (0m 13s)
iter: 15104 35% (0m 14s)
iter: 16128 37% (0m 15s)
iter: 17024 39% (0m 15s)
iter: 18048 42% (0m 16s)
iter: 19072 44% (0m 17s)
iter: 20096 47% (0m 18s)
iter: 21120 49% (0m 19s)
iter: 22016 51% (0m 20s)
iter: 23040 53% (0m 21s)
iter: 24064 56% (0m 22s)
iter: 25088 58% (0m 23s)
iter: 26112 61% (0m 24s)
iter: 27008 63% (0m 24s)
iter: 28032 65% (0m 25s)
iter: 29056 68% (0m 26s)
iter: 30080 70% (0m 27s)
iter: 31104 72% (0m 28s)
iter: 32128 75% (0m 29s)
iter: 33024 77% (0m 30s)
iter: 34048 79% (0m 31s)
iter: 35072 82% (0m 33s)
iter: 36096 84% (0m 33s)
iter: 37120 86% (0m 34s)
iter: 38016 89% (0m 35s)
iter: 39040 91% (0m 36s)
iter: 40064 93% (0m 37s)
it

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:37: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:38: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


(tensor(0.7104),
 tensor(0.0134, device='cuda:0'),
 tensor(0.0368, device='cuda:0'),
 tensor(0.0082, device='cuda:0'))

In [159]:
import torch.optim as optim
learning_rate = 0.0005
max_epochs = 10
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
def train(input_tensor, label_tensor):
  hidden = model.initHidden(batch_size=batch_size)
  model.zero_grad()
  label_tensor = torch.tensor([label_tensor])

  if cudaEnabled: 
    label_tensor = label_tensor.cuda()
    input = input_tensor.cuda()

  output, hidden = model(input, hidden)
  final_output = output[:, -1, :].T
  loss = weighted_binary_cross_entropy(final_output , label_tensor, weights=class_weights)
  loss.backward()
  optimizer.step()
  return final_output, loss.item()
  

In [160]:
current_loss = 0
all_lstm_losses = []
all_eval_losses = []
max_epochs = 10
n_iter = max_epochs * len(train_dataset)

In [ ]:
f1 = F1Score(num_classes=1).cuda()
start = time.time()
for epoch in range(max_epochs):
    # Training
    print_count = 1
    plot_count = 1
    for iter, (local_batch, local_labels) in enumerate(train_loader):
        # Model computations
        preds, loss = train(local_batch, local_labels)
        current_loss += loss
        if iter * batch_size > print_every * print_count:
          labels = torch.tensor(local_labels).cuda()
          correct = f'f1 score: {f1(preds[0], labels)}' 
          print(f'epoch: {epoch} iter: {iter* batch_size} {math.floor(iter* batch_size/len(train_dataset) * 100)}% ({timeSince(start)}) {round(loss, 4)} / {correct}')
          print_count += 1
        if iter * batch_size > plot_every * plot_count:
          eval_loss, f1_score, precision_score, recall_score = evaluate(model)
          all_eval_losses.append(eval_loss)
          all_lstm_losses.append(current_loss/ plot_every)
          current_loss = 0
          plot_count += 1
    print(50*'-' + 'training' + 50*'-')
torch.save(model.state_dict(), f'/content/drive/MyDrive/models/model_{model_name}_{mode}.pth')
    

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:61: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


epoch: 0 iter: 1024 0% (0m 1s) 0.5337 / f1 score: 0.03100775182247162
epoch: 0 iter: 2048 1% (0m 2s) 0.7883 / f1 score: 0.07633588463068008
epoch: 0 iter: 3072 1% (0m 3s) 0.875 / f1 score: 0.09090908616781235
epoch: 0 iter: 4096 2% (0m 4s) 0.5369 / f1 score: 0.03100775182247162
epoch: 0 iter: 5120 2% (0m 5s) 0.6169 / f1 score: 0.04724409431219101
epoch: 0 iter: 6016 3% (0m 6s) 0.7087 / f1 score: 0.04615384712815285
epoch: 0 iter: 7040 3% (0m 7s) 1.2218 / f1 score: 0.145985409617424
epoch: 0 iter: 8064 4% (0m 8s) 0.5321 / f1 score: 0.03100775182247162
epoch: 0 iter: 9088 4% (0m 9s) 0.7928 / f1 score: 0.0
epoch: 0 iter: 10112 5% (0m 10s) 0.7007 / f1 score: 0.0
epoch: 0 iter: 11008 5% (0m 11s) 0.7056 / f1 score: 0.0
epoch: 0 iter: 12032 6% (0m 12s) 0.6129 / f1 score: 0.0
epoch: 0 iter: 13056 6% (0m 13s) 1.0618 / f1 score: 0.0
epoch: 0 iter: 14080 7% (0m 14s) 0.6175 / f1 score: 0.04651162773370743
epoch: 0 iter: 15104 7% (0m 15s) 0.7031 / f1 score: 0.0625
epoch: 0 iter: 16128 8% (0m 17s) 0

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


iter: 128 0% (0m 0s)
iter: 1024 2% (0m 1s)
iter: 2048 4% (0m 1s)
iter: 3072 7% (0m 2s)
iter: 4096 9% (0m 3s)
iter: 5120 11% (0m 4s)
iter: 6016 14% (0m 5s)
iter: 7040 16% (0m 6s)
iter: 8064 18% (0m 7s)
iter: 9088 21% (0m 8s)
iter: 10112 23% (0m 9s)
iter: 11008 25% (0m 10s)
iter: 12032 28% (0m 11s)
iter: 13056 30% (0m 12s)
iter: 14080 32% (0m 12s)
iter: 15104 35% (0m 13s)
iter: 16128 37% (0m 14s)
iter: 17024 39% (0m 15s)
iter: 18048 42% (0m 16s)
iter: 19072 44% (0m 17s)
iter: 20096 47% (0m 18s)
iter: 21120 49% (0m 19s)
iter: 22016 51% (0m 20s)
iter: 23040 53% (0m 21s)
iter: 24064 56% (0m 22s)
iter: 25088 58% (0m 23s)
iter: 26112 61% (0m 23s)
iter: 27008 63% (0m 24s)
iter: 28032 65% (0m 25s)
iter: 29056 68% (0m 26s)
iter: 30080 70% (0m 27s)
iter: 31104 72% (0m 28s)
iter: 32128 75% (0m 29s)
iter: 33024 77% (0m 30s)
iter: 34048 79% (0m 31s)
iter: 35072 82% (0m 32s)
iter: 36096 84% (0m 33s)
iter: 37120 86% (0m 34s)
iter: 38016 89% (0m 34s)
iter: 39040 91% (0m 35s)
iter: 40064 93% (0m 36s)
it

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:37: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:38: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


f1 score: 0.005912786349654198
precison score: 0.5
recall score: 0.0029739777091890574
epoch: 0 iter: 51072 25% (1m 32s) 0.8827 / f1 score: 0.0
epoch: 0 iter: 52096 26% (1m 33s) 0.9751 / f1 score: 0.0
epoch: 0 iter: 53120 26% (1m 34s) 0.619 / f1 score: 0.04615384712815285
epoch: 0 iter: 54016 27% (1m 35s) 0.7922 / f1 score: 0.07575757801532745
epoch: 0 iter: 55040 27% (1m 36s) 0.5276 / f1 score: 0.03539822995662689
epoch: 0 iter: 56064 28% (1m 37s) 0.7069 / f1 score: 0.05042016878724098
epoch: 0 iter: 57088 28% (1m 38s) 0.4645 / f1 score: 0.0
epoch: 0 iter: 58112 29% (1m 39s) 0.7048 / f1 score: 0.07207206636667252
epoch: 0 iter: 59008 29% (1m 40s) 0.6127 / f1 score: 0.0
epoch: 0 iter: 60032 30% (1m 42s) 0.6169 / f1 score: 0.0
epoch: 0 iter: 61056 30% (1m 43s) 0.4389 / f1 score: 0.0
epoch: 0 iter: 62080 31% (1m 44s) 0.6235 / f1 score: 0.0
epoch: 0 iter: 63104 31% (1m 45s) 1.3199 / f1 score: 0.0
epoch: 0 iter: 64128 32% (1m 46s) 0.5306 / f1 score: 0.03305785357952118
epoch: 0 iter: 65024

In [ ]:
evaluate(model, is_test=True)

--------------------------------------------------evaluating--------------------------------------------------


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:61: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


iter: 128 0% (0m 0s)
iter: 1024 3% (0m 1s)
iter: 2048 7% (0m 2s)
iter: 3072 11% (0m 2s)
iter: 4096 14% (0m 3s)
iter: 5120 18% (0m 4s)
iter: 6016 21% (0m 5s)
iter: 7040 25% (0m 6s)
iter: 8064 29% (0m 7s)
iter: 9088 33% (0m 8s)
iter: 10112 36% (0m 10s)
iter: 11008 40% (0m 12s)
iter: 12032 43% (0m 13s)
iter: 13056 47% (0m 14s)
iter: 14080 51% (0m 14s)
iter: 15104 55% (0m 15s)
iter: 16128 58% (0m 16s)
iter: 17024 62% (0m 17s)
iter: 18048 65% (0m 18s)
iter: 19072 69% (0m 19s)
iter: 20096 73% (0m 20s)
iter: 21120 76% (0m 21s)
iter: 22016 80% (0m 22s)
iter: 23040 83% (0m 23s)
iter: 24064 87% (0m 24s)
iter: 25088 91% (0m 25s)
iter: 26112 95% (0m 25s)
iter: 27008 98% (0m 26s)
f1 score: 0.8919824957847595
precison score: 0.8116350173950195
recall score: 0.9899857044219971


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:37: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:38: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


(tensor(0.0640),
 tensor(0.8920, device='cuda:0'),
 tensor(0.8116, device='cuda:0'),
 tensor(0.9900, device='cuda:0'))

In [ ]:
lstm_losses = []